In [10]:
import sys
sys.path.append('../')
import os
from models.UNet.datasets import ImageNameDataset, get_size
from torch.utils.data import random_split
import torch
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

import shutil

import math

In [11]:
# ImageFolder dataset alternative
#from torchvision.datasets import ImageFolder
#from torchvision.transforms import Compose, ToTensor, Resize
#from torch.utils.data import Subset

#dataset = ImageFolder('C:\Datasets\lcms-dataset', transform=Compose([Resize((224,224)),ToTensor()]))
#def train_val_dataset(dataset, val_split=0.25):
#    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
#    datasets = {}
#    datasets['train'] = Subset(dataset, train_idx)
#    datasets['val'] = Subset(dataset, val_idx)
#    return datasets

### set params.

In [12]:
dataset_folder = '../data/2024-04-21-dataset/standard/formatted'
train_size = 0.90
seed = 42
batch_size = 1
shuffle = False

dataset = ImageNameDataset(dataset_folder, sort = True)

In [13]:
images = [sample['image'] for sample in dataset]
masks = [sample['mask'] for sample in dataset]

### casual (scikit-learn) train-test-split

In [18]:
images_train, images_test, masks_train, masks_test = train_test_split(images, masks,
                                                                      train_size=train_size,
                                                                      random_state = seed,
                                                                      shuffle = shuffle)
print(len(images_train),len(images_test))

1699 300


In [19]:
# can't make it to not random 
#train_set, val_set = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(0))

In [14]:
def middle_split(images, masks, train_size):
    test_images_count = int(len(images)*(1-train_size))
    X = Y = math.sqrt(test_images_count)
    if type(Y)==float:
        X = int(X)
        Y = test_images_count // int(Y)

    max_x, max_y = os.path.splitext(os.path.basename(images[-1]))[0].split('_')[2:4]
    mid_x = int(int(max_x)/2)
    mid_y = int(int(max_y)/2)
    x_start = mid_x//2 - X//2
    y_start = mid_y//2 - Y//2

    images_test=[]
    masks_test=[]

    x_end = x_start+X
    y_end = y_start+Y
    for x in range(x_start,x_end+1):
        for y in range(y_start,y_end+1):
            image_name = images[0][:-17]+f'tile_tif_{x}_{y}.png'
            mask_name = images[0][:-24]+f'masks/tile_shp_{x}_{y}.png'
            if image_name in images:
                images.remove(image_name)
                images_test.append(image_name)
                masks.remove(mask_name)
                masks_test.append(mask_name)
                
    return images, images_test, masks, masks_test
    
images_train, images_test, masks_train, masks_test = middle_split(images,masks,train_size)

print(len(images_train),len(images_test))
print(len(masks_train),len(masks_test))


1970 29
1970 29


### get the actual split ratio

In [6]:
train_size, test_size, actual_get_size = get_size(dataset,train_size)
print(train_size, test_size, actual_get_size)

1799 200 0.8999499749874937


### create folders

In [15]:
train_folder = '../data/2024-04-21-dataset/standard/train'
test_folder = '../data/2024-04-21-dataset/standard/test'

train_images_folder = os.path.join(train_folder,'images')
train_masks_folder = os.path.join(train_folder,'masks')
test_images_folder = os.path.join(test_folder,'images')
test_masks_folder = os.path.join(test_folder,'masks')

In [16]:
os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(train_masks_folder, exist_ok=True)
os.makedirs(test_images_folder, exist_ok=True)
os.makedirs(test_masks_folder, exist_ok=True)

### copy files

In [17]:
for path in images_train:
    shutil.copy(path,train_images_folder)
for path in masks_train:
    shutil.copy(path,train_masks_folder)
for path in images_test:
    shutil.copy(path,test_images_folder)
for path in masks_test:
    shutil.copy(path,test_masks_folder)